In [7]:
import cv2
import numpy as np
import os

In [8]:
dataset_path="./data/"
faceData=[]
labels=[]
nameMap={}
classId=0
offset=20

In [9]:
for f in os.listdir(dataset_path):
    if f.endswith(".npy"):
        nameMap[classId]=f[:-4]
        # X- values
        dataItem=np.load(dataset_path+f)
        m=dataItem.shape[0]
        print(f, dataItem.shape)
        faceData.append(dataItem)
        # Y- values
        target=classId* np.ones((m,))
        classId+=1
        labels.append(target)


XT=np.concatenate(faceData,axis=0)
yT=np.concatenate(labels,axis=0).reshape((-1,1))

# print(XT.shape)
# print(yT.shape)
# print(nameMap)

Gaurav.npy (21, 30000)


In [10]:
def dist(p,q):
    return np.sqrt(np.sum((p - q)**2))

def knn(X,y,xt,k=5):

    m = X.shape[0]
    dlist = []

    for i in range(m):
        d = dist(X[i],xt)
        dlist.append((d,int(y[i])))

    dlist = sorted(dlist)
    dlist = np.array(dlist[:k])
    labels = dlist[:,1]

    labels, cnts = np.unique(labels,return_counts=True)
    idx = cnts.argmax()
    pred = labels[idx]

    return int(pred)

In [11]:
# Predictions

cam=cv2.VideoCapture(0)

model = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')

In [12]:
#  Read image from camera Object
while True:
    success,img=cam.read()
    if not success:
        print("Reading Camera Failed")
    
    faces = model.detectMultiScale(img, 1.3, 5)
    # render a box around each face and predicts its name
    for f in faces:
        f=faces[-1]
        x,y,w,h=f
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    
        cropped_face=img[y-offset:y+h+offset,x-offset:x+w+offset]
        cropped_face=cv2.resize(cropped_face,(100,100))
        

        # predict the name using KNN
        classPredicted=knn(XT,yT,cropped_face.flatten())
        # Name
        namePredicted=nameMap[classPredicted]
        cv2.putText(img,namePredicted,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,25,0),2)
    cv2.imshow("Prediction Window",img)
    key=cv2.waitKey(1) 
    if key==ord('q'):
        break

# release the camera and destroy the window
cam.release()
cv2.destroyAllWindows()

C:\Users\Offic\AppData\Local\Temp\ipykernel_25500\2550623409.py:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  dlist.append((d,int(y[i])))
